<a href="https://colab.research.google.com/github/jamagiwa/Kaggle/blob/Explore_und_practice/GBM_scRNAseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GBM scRNAseqデータを用いてEDAの練習する

What: scRNAseqデータを用いて解析の手法を学ぶ

---

Why: 遺伝子発現、epigenome、proteomeデータを自由に扱えるようになりたいため

---

Result


In [8]:
#データのダウンロード
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("reminho/human-glioblastoma-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/human-glioblastoma-dataset


In [9]:
#ファイル名の確認
import os
print(os.listdir(path))

['barcodes.tsv', 'matrix.mtx', 'genes.tsv']


barcodes.tsv...各細胞のID, matrix.mtx...実際の遺伝子発現data(ほとんどのgeneが0なので特殊な形式で効率よく保存している。), genes.tsv...遺伝子名のリスト

3つのファイルを読み込む

In [29]:
import pandas as pd
import scipy.io

#ファイルパスを設定
path = "/kaggle/input/human-glioblastoma-dataset"
barcodes_file = f"{path}/barcodes.tsv"
gene_file = f"{path}/genes.tsv"
matrix_file = f"{path}/matrix.mtx"

#matrix/mtxを読み込む
matrix = scipy.io.mmread(matrix_file).T#mmread関数でmtxを読み込む

#barcode.tsvとgenes.tsvを読み込む
barcodes = pd.read_csv(barcodes_file, header = None).iloc[:, 0].values#iloc[:, 0].valuesで最初の行をインデックスに変換
genes = pd.read_csv(gene_file, header = None).iloc[:, 0].values

#pandasのDataFrameにまとめる
df = pd.DataFrame(data=matrix.toarray(), index=barcodes, columns=genes)

print(df.head())


                    ENSG00000078369\tGNB1\tGene Expression  \
AAACCCAAGGCGATAC-1                                       1   
AAACCCAAGGCTGTAG-1                                       0   
AAACCCACAAGTCCCG-1                                       5   
AAACCCACAGATGCGA-1                                       2   
AAACCCACAGGTGAGT-1                                       4   

                    ENSG00000078900\tTP73\tGene Expression  \
AAACCCAAGGCGATAC-1                                       0   
AAACCCAAGGCTGTAG-1                                       0   
AAACCCACAAGTCCCG-1                                       0   
AAACCCACAGATGCGA-1                                       0   
AAACCCACAGGTGAGT-1                                       0   

                    ENSG00000116288\tPARK7\tGene Expression  \
AAACCCAAGGCGATAC-1                                        0   
AAACCCAAGGCTGTAG-1                                        0   
AAACCCACAAGTCCCG-1                                       13   
AA